# 🚀 mHealth Privacy Experiments - Optimized Pipeline

**Simplified training pipeline using project scripts**

- ✅ Setup (Drive, Clone, Dependencies)
- ✅ Update/Reclonar anytime
- ✅ Configure & Run experiments
- ✅ Load & Analyze results

**Author** Eduardo Barbosa, Filipe Correia, Vasco Fernandes  
**Version:** 2.0 (Optimized)  
**Date:** October 2025

## 1️⃣ SETUP: Mount Drive & Clone Repository

In [ ]:
from google.colab import drive
from pathlib import Path
import os
import sys
import subprocess
import shutil

# Mount Drive
drive.mount('/content/drive', force_remount=True)

# Setup paths
DRIVE_BASE = Path('/content/drive/MyDrive')
DATA_DRIVE = DRIVE_BASE / 'mhealth-data/data'
PROJECT_DIR = Path('/content/mhealth-data-privacy')
REPO_URL = 'https://github.com/vasco-fernandes21/mhealth-data-privacy.git'

print('📁 Paths Configuration')
print(f'  Project: {PROJECT_DIR}')
print(f'  Data (Drive): {DATA_DRIVE}')
print(f'  Data exists: {DATA_DRIVE.exists()}')

# Clone or update repository
if not PROJECT_DIR.exists():
    print(f'\n📥 Cloning repository...')
    result = subprocess.run(['git', 'clone', REPO_URL, str(PROJECT_DIR)],
                          capture_output=True, text=True)
    if result.returncode == 0:
        print('✅ Repository cloned')
    else:
        print(f'❌ Clone failed: {result.stderr}')
else:
    print('✅ Project already exists')
    os.chdir(PROJECT_DIR)
    result = subprocess.run(['git', 'pull'], capture_output=True, text=True)
    if result.returncode == 0:
        print('✅ Repository updated')

os.chdir(PROJECT_DIR)

# Create symlink for data
data_link = PROJECT_DIR / 'data'
if not data_link.exists() and DATA_DRIVE.exists():
    print(f'\n🔗 Creating symlink: {data_link} → {DATA_DRIVE}')
    os.symlink(DATA_DRIVE, data_link)
    print('✅ Symlink created')
elif data_link.is_symlink():
    print(f'\n🔗 Symlink already exists')

print(f'\n✅ Setup complete!')
print(f'   Working directory: {PROJECT_DIR}')

Mounted at /content/drive
📁 Paths Configuration
  Project: /content/mhealth-data-privacy
  Data (Drive): /content/drive/MyDrive/mhealth-data/data
  Data exists: True

📥 Cloning repository...
✅ Repository cloned

🔗 Creating symlink: /content/mhealth-data-privacy/data → /content/drive/MyDrive/mhealth-data/data
✅ Symlink created

✅ Setup complete!
   Working directory: /content/mhealth-data-privacy


## 2️⃣ SETUP: Install Dependencies

In [ ]:
!pip install -q opacus scikit-learn pyyaml tqdm pyedflib mne

import torch
import json
import time
from datetime import datetime
import pandas as pd
import numpy as np

# Setup device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
RESULTS_DIR = PROJECT_DIR / 'experiments'
RESULTS_DIR.mkdir(exist_ok=True)

print('✅ Dependencies installed')
print(f'\n📊 Environment:')
print(f'   Device: {DEVICE}')
if DEVICE == 'cuda':
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 146.6 MB/s eta 0:00:00
✅ Dependencies installed

📊 Environment:
   Device: cuda
   GPU: Tesla T4
   VRAM: 14.7 GB


## 3️⃣ UPDATE: Repository (Anytime)

In [ ]:
# ============================================================================
# 🔄 REPOSITORY UPDATE
# ============================================================================

# ⚙️ CONFIGURE ACTION HERE
UPDATE_ACTION = 'reset'  # Options: 'pull', 'reset', 'reclone'

# ============================================================================

def execute_update():
    """Execute repository update based on UPDATE_ACTION variable."""

    action = UPDATE_ACTION.lower().strip()

    print(f"\n{'='*70}")
    print(f"🔄 REPOSITORY UPDATE: {action.upper()}")
    print(f"{'='*70}\n")

    try:
        if action == 'pull':
            os.chdir(PROJECT_DIR)
            print("📥 Pulling latest changes...")
            result = subprocess.run(['git', 'pull'], capture_output=True, text=True)

            if result.returncode == 0:
                print('✅ Pull successful')
                if result.stdout:
                    print(result.stdout)
            else:
                print(f'❌ Pull failed: {result.stderr}')

        elif action == 'reset':
            os.chdir(PROJECT_DIR)
            print("🔄 Resetting to origin/main...")
            result = subprocess.run(['git', 'reset', '--hard', 'origin/main'],
                                  capture_output=True, text=True)

            if result.returncode == 0:
                print('✅ Reset successful')
                if result.stdout:
                    print(result.stdout)
            else:
                print(f'❌ Reset failed: {result.stderr}')

        elif action == 'reclone':
            print('⚠️  DELETING and RECLONING repository\n')

            print('🗑️  Deleting old project...')
            if PROJECT_DIR.exists():
                shutil.rmtree(PROJECT_DIR)
            print('✅ Deleted')

            print('📥 Cloning repository...')
            result = subprocess.run(['git', 'clone', REPO_URL, str(PROJECT_DIR)],
                                  capture_output=True, text=True)

            if result.returncode == 0:
                print('✅ Clone successful')

                print('🔗 Creating symlink...')
                os.symlink(DATA_DRIVE, PROJECT_DIR / 'data')
                print('✅ Symlink created')

                os.chdir(PROJECT_DIR)
                print('✅ Reclone complete')
            else:
                print(f'❌ Clone failed: {result.stderr}')

        elif action == 'skip':
            print('⏭️  Skipped (UPDATE_ACTION = "skip")')

        else:
            print(f'❌ Unknown action: {action}')
            print('Available: pull, reset, reclone, skip')

        print(f"\n{'='*70}\n")

    except Exception as e:
        print(f'❌ Error: {e}\n')

# AUTO-EXECUTE based on UPDATE_ACTION
execute_update()


🔄 REPOSITORY UPDATE: RESET

🔄 Resetting to origin/main...
✅ Reset successful
HEAD is now at 363d32f test #2





## 4️⃣ CONFIG: Experiment Settings

In [ ]:
# ============================================================================
# ⚙️ CONFIGURE EXPERIMENTS HERE
# ============================================================================

# Which scenarios to run
SCENARIO = 'baseline'           # Options: baseline, dp, fl, fl_dp, all

# Filters
TAGS = ''                  # e.g., 'tier1' or '' (empty = no filter)
DATASETS = 'wesad'              # e.g., 'wesad', 'sleep-edf' or '' (empty = all)
N_EXPERIMENTS = None            # e.g., 3 (limit to first N) or None (all)

# Execution
DRY_RUN = False                 # True = show commands only
AUTO_MODE = True                # True = skip confirmation

# ============================================================================

print('⚙️ CONFIGURATION')
print('='*70)
print(f'Scenario: {SCENARIO}')
print(f'Tags: {TAGS if TAGS else "(none)"}')
print(f'Datasets: {DATASETS if DATASETS else "(all)"}')
print(f'Limit experiments: {N_EXPERIMENTS if N_EXPERIMENTS else "(no limit)"}')
print(f'Dry run: {DRY_RUN}')
print(f'Auto mode: {AUTO_MODE}')
print('='*70)

⚙️ CONFIGURATION
Scenario: baseline
Tags: (none)
Datasets: wesad
Limit experiments: (no limit)
Dry run: False
Auto mode: True


## 5️⃣ EXECUTE: Run Experiments

In [ ]:
# Build command com -u (unbuffered)
cmd = [
    'python', '-u',  # ← ADICIONA ISTO (unbuffered output)
    'experiments/run_experiments.py',
    '--scenario', SCENARIO,
    '--device', DEVICE,
]

# Add filters
if TAGS:
    cmd += ['--tags', TAGS]
if DATASETS:
    cmd += ['--datasets', DATASETS]
if N_EXPERIMENTS:
    cmd += ['--n_experiments', str(N_EXPERIMENTS)]
if AUTO_MODE:
    cmd += ['--auto']

print('\n' + '='*70)
print('🚀 RUNNING EXPERIMENTS')
print('='*70)
print(f'\nCommand: {" ".join(cmd)}\n')
print('📊 LIVE OUTPUT:')
print('='*70 + '\n')

# Execute with streaming output
process = subprocess.Popen(
    cmd,
    cwd=str(PROJECT_DIR),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,  # Line buffering
    universal_newlines=True
)

# Stream output in real-time
for line in process.stdout:
    print(line, end='', flush=True)

# Wait for completion
return_code = process.wait()

print('\n' + '='*70)
if return_code == 0:
    print('✅ Experiments completed successfully')
else:
    print(f'❌ Experiments failed (exit code: {return_code})')
print('='*70)


🚀 RUNNING EXPERIMENTS

Command: python -u experiments/run_experiments.py --scenario baseline --device cuda --datasets wesad --auto

📊 LIVE OUTPUT:

Device: cuda

Loaded 6 experiments

Will run: 3 experiments


Running 3 experiments...

[1/3]

baseline_wesad_run1 | wesad | baseline | seed=42

Config: batch_size=256, lr=0.0003, epochs=80

Loading data...
Loading WESAD from data/processed/wesad...
✅ Loaded:
   Train: (1217, 14, 1024)
   Val:   (407, 14, 1024)
   Test:  (408, 14, 1024)
   Classes: ['non-stress', 'stress']

Shapes: train=(1217, 14, 1024), val=(407, 14, 1024), test=(408, 14, 1024)

Creating model...
Model: 118,530 parameters

Epoch 001: train_loss=0.6328 train_acc=0.6606 | val_loss=0.5455 val_acc=0.7346
Epoch 002: train_loss=0.5131 train_acc=0.7831 | val_loss=0.4793 val_acc=0.7494
Epoch 003: train_loss=0.4467 train_acc=0.8348 | val_loss=0.4432 val_acc=0.8034
Epoch 004: train_loss=0.3937 train_acc=0.8759 | val_loss=0.4218 val_acc=0.8452
Epoch 005: train_loss=0.3652 train_acc

## 6️⃣ RESULTS: Load & Analyze

In [ ]:
# Load results
print(RESULTS_DIR)
results_file = RESULTS_DIR / 'results_log.json'
print(results_file)


if results_file.exists():
    with open(results_file) as f:
        results = json.load(f)

    print('\n' + '='*70)
    print('📊 RESULTS SUMMARY')
    print('='*70)

    print(f'\nTimestamp: {results["timestamp"]}')
    print(f'Total experiments: {results["total"]}')
    print(f'Successful: {results["successful"]} ✅')
    print(f'Failed: {results["failed"]} ❌')
    print(f'Success rate: {results["successful"]/results["total"]*100:.1f}%')
    print(f'Total time: {results["total_time_hours"]:.2f} hours')

    # Convert to DataFrame
    df = pd.DataFrame(results['results'])

    print(f'\nDetailed Results:')
    print(df[['name', 'method', 'dataset', 'seed', 'success', 'time_seconds']].to_string(index=False))

    print(f'\n{"="*70}')
else:
    print('❌ Results file not found')

/content/mhealth-data-privacy/experiments
/content/mhealth-data-privacy/experiments/results_log.json

📊 RESULTS SUMMARY

Timestamp: 2025-10-28T19:45:15.506866


KeyError: 'total'

## 7️⃣ DOWNLOAD: Results

In [ ]:
from google.colab import files

print('\n📥 DOWNLOADING RESULTS')
print('='*70)

# Download JSON log
if results_file.exists():
    print(f'\nDownloading: experiments_log.json')
    files.download(str(results_file))

# Create archive of all results
if RESULTS_DIR.exists() and list(RESULTS_DIR.glob('*')):
    print(f'Creating archive...')
    archive = shutil.make_archive('/tmp/mhealth_results', 'zip', RESULTS_DIR)
    print(f'Downloading: mhealth_results.zip')
    files.download(archive)

print(f'\n✅ Download complete!')
print('='*70)

## 8️⃣ UTILITIES: Helper Functions

In [ ]:
def show_scenarios():
    """List available scenarios."""
    import yaml

    scenarios_dir = PROJECT_DIR / 'experiments' / 'scenarios'
    yaml_files = list(scenarios_dir.glob('*.yaml'))

    print(f'\n📋 Available Scenarios')
    for f in sorted(yaml_files):
        with open(f) as fp:
            data = yaml.safe_load(fp)
            n_exp = len(data.get('experiments', {}))
            print(f'  - {f.stem}: {n_exp} experiments')


def quick_test():
    """Run quick test (first 3 experiments)."""
    print('\n🚀 Quick Test: Running first 3 baseline experiments on WESAD')

    cmd = [
        'python', 'experiments/run_experiments.py',
        '--scenario', 'baseline',
        '--tags', 'tier1',
        '--datasets', 'wesad',
        '--n_experiments', '1',
        '--device', DEVICE,
        '--auto'
    ]

    subprocess.run(cmd, cwd=str(PROJECT_DIR))


def show_last_results():
    """Show last run results."""
    if results_file.exists():
        with open(results_file) as f:
            results = json.load(f)

        df = pd.DataFrame(results['results'])
        print('\n📊 Last Run Results')
        print(df.to_string())
    else:
        print('No results available')


print('✅ Utilities loaded')
print('\nUsage:')
print('  show_scenarios()      # List all scenarios')
print('  quick_test()          # Run quick test')
print('  show_last_results()   # Show last results')
print('  quick_update_menu()   # Update repository')

## 9️⃣ EXAMPLES

In [ ]:
# ============================================================================
# EXAMPLE CONFIGURATIONS
# ============================================================================

print("📝 Example Configurations (Run in Section 4):\n")

print("Example 1: Quick test (1 experiment)")
print("""
SCENARIO = 'baseline'
TAGS = 'tier1'
DATASETS = 'wesad'
N_EXPERIMENTS = 1
AUTO_MODE = True
""")

print("\nExample 2: All baseline on WESAD")
print("""
SCENARIO = 'baseline'
TAGS = 'tier1'
DATASETS = 'wesad'
N_EXPERIMENTS = None
AUTO_MODE = True
""")

print("\nExample 3: DP experiments")
print("""
SCENARIO = 'dp'
TAGS = 'tier1'
DATASETS = 'wesad'
N_EXPERIMENTS = None
AUTO_MODE = True
""")

print("\nExample 4: All scenarios (long run)")
print("""
SCENARIO = 'all'
TAGS = 'tier1'
DATASETS = None
N_EXPERIMENTS = None
AUTO_MODE = True
""")

print("\nExample 5: Dry run (see commands)")
print("""
SCENARIO = 'baseline'
TAGS = 'tier1'
DATASETS = 'wesad'
DRY_RUN = True
AUTO_MODE = True
""")

In [ ]:
# Download da results
import shutil
from google.colab import files

# Caminho da pasta a compactar
pasta_origem = "/content/mhealth-data-privacy/results"

# Caminho do zip de destino
zip_destino = "/content/results.zip"

# Criar zip
shutil.make_archive(base_name="/content/results", format='zip', root_dir=pasta_origem)

# Fazer download
files.download(zip_destino)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>